In [1]:
import pandas as pd
import seaborn as sns

C:\Users\VIJESH\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [59]:
train=pd.read_csv('train_s3TEQDk.csv')
test=pd.read_csv('test_mSzZ8RL.csv')

In [60]:
train.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,NNVBBKZB,Female,73,RG268,Other,X3,43,No,1045696,No,0
1,IDD62UNG,Female,30,RG277,Salaried,X1,32,No,581988,No,0
2,HD3DSEMC,Female,56,RG268,Self_Employed,X3,26,No,1484315,Yes,0
3,BF3NC7KV,Male,34,RG270,Salaried,X1,19,No,470454,No,0
4,TEASRWXV,Female,30,RG282,Salaried,X1,33,No,886787,No,0


In [61]:
import numpy as np

In [68]:
train['Avg_Account_Balance']=np.log1p(train['Avg_Account_Balance'])
test['Avg_Account_Balance']=np.log1p(test['Avg_Account_Balance'])


In [63]:
train=train.fillna('Yes')
test=test.fillna('Yes')

In [65]:
num_cols=['Age','Vintage']
cat_cols=[col for col in train.columns if col  not in num_cols +['ID','Is_Lead']]

In [66]:
from sklearn.preprocessing import LabelEncoder
for col in cat_cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])


In [12]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()
encoded =encoder.fit_transform(train[cat_cols]).toarray()
encoded=pd.DataFrame(encoded)
encoded_test=encoder.transform(test[cat_cols]).toarray()
encoded_test=pd.DataFrame(encoded_test)

In [13]:
#encoded.head()

In [14]:
from sklearn.preprocessing import MinMaxScaler
for col in num_cols:
    train[col]=MinMaxScaler().fit_transform(train[col].values.reshape(-1,1))
    test[col]=MinMaxScaler().fit_transform(test[col].values.reshape(-1,1))

In [15]:
train.head()

,ID,Gender,Age,Region_Code,Occupation,Channel_Code,Vintage,Credit_Product,Avg_Account_Balance,Is_Active,Is_Lead
0,NNVBBKZB,0,0.806452,18,1,2,0.281250,0,0.630865,0,0
1,IDD62UNG,0,0.112903,27,2,0,0.195312,0,0.536510,0,0
2,HD3DSEMC,0,0.532258,18,3,2,0.148438,0,0.687265,1,0
3,BF3NC7KV,1,0.177419,20,2,0,0.093750,0,0.502253,0,0
4,TEASRWXV,0,0.112903,32,2,0,0.203125,0,0.604324,0,0


In [16]:
encoded[num_cols]=train[num_cols]
encoded_test[num_cols]=test[num_cols]

In [18]:
from sklearn.metrics import classification_report, confusion_matrix,accuracy_score
from sklearn.metrics import roc_auc_score
X = encoded
y = train.Is_Lead

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,stratify=y)

<IPython.core.display.Javascript object>

In [818]:
from lightgbm import LGBMClassifier

In [819]:
model = LGBMClassifier()
model.fit(X_train, y_train)
pred=model.predict(X_test)
accuracy_score(y_test,pred)

0.8249262386814529

In [820]:
roc_auc_score(y_test, pred)

0.7386926810352978

In [823]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, X, y, cv=10,scoring = 'roc_auc')
print(scores,scores.mean())

[0.85058906 0.85016487 0.85080382 0.85353797 0.84686813 0.85218918
 0.85080316 0.85045794 0.85376139 0.849916  ] 0.8509091502572679


In [824]:
model.fit(X,y)

LGBMClassifier()

In [825]:
prediction=model.predict(encoded_test)

In [826]:
file=pd.DataFrame()
file['ID']=test['ID']
file['Is_Lead']=prediction

In [827]:
file.to_csv('Final.csv',index=False)